<a href="https://colab.research.google.com/github/JosDeleon/Seminario-Profesional-I/blob/master/Laboratorio07_17004895.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only works in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [0]:
# cnn model for the har dataset
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical

In [0]:
from google.colab import drive
drive.mount('/content/drive')
dataset_path = "/content/drive/My Drive/SP1_2020/HAR with CNN/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# run this only once to save dataset to drive
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/HAR_Smartphones.zip -P 

wget: option requires an argument -- 'P'
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [0]:
ls -l "/content/drive/My Drive/SP1_2020/HAR with CNN/"

ls: cannot access '/content/drive/My Drive/SP1_2020/HAR with CNN/': No such file or directory


In [0]:
!unzip -q "/content/drive/My Drive/SP1_2020/HAR with CNN/HAR_Smartphones.zip" -d "/content/drive/My Drive/SP1_2020/HAR with CNN/"

replace /content/drive/My Drive/SP1_2020/HAR with CNN/UCI HAR Dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/HAR with CNN/__MACOSX/UCI HAR Dataset/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/HAR with CNN/UCI HAR Dataset/activity_labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/HAR with CNN/__MACOSX/UCI HAR Dataset/._activity_labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/HAR with CNN/UCI HAR Dataset/features.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/HAR with CNN/__MACOSX/UCI HAR Dataset/._features.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import datetime, os

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
# checkpoint
filepath = dataset_path + "output.best.h5"
checkpoint_callback = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min',save_freq='epoch')

# tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1,profile_batch=0)

# callbacks
callbacks_list = [checkpoint_callback,tensorboard_callback]

In [0]:
# Run Tensorboad for monitoring
%tensorboard --logdir logs

In [0]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()

    print("\nShape dimensions:\n")
    for name in filenames:
        data = load_file(prefix + name)
        print(np.array(loaded).shape)
        loaded.append(data)
      
     
    # stack group so that features are the 3rd dimension
    print(np.array(loaded).shape)
    loaded = np.dstack(loaded)
    print(np.array(loaded).shape)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    print("\nOne Hot Encoding: \n",trainy)
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print("\n",trainy)
    return trainX, trainy, testX, testy

In [0]:
model = Sequential()

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    global model
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose,callbacks=callbacks_list)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [0]:
trainX, trainy, testX, testy = load_dataset("/content/drive/My Drive/SP1_2020/HAR with CNN/")


Shape dimensions:



FileNotFoundError: ignored

In [0]:
score = evaluate_model(trainX, trainy, testX, testy)
score = score * 100.0
print('>Accuracy: %.3f' % (score))

Epoch 1/10
228/230 [============================>.] - ETA: 0s - loss: 0.3681 - accuracy: 0.8481
Epoch 00001: loss improved from inf to 0.36575, saving model to /content/drive/My Drive/SP1_2020/HAR with CNN/output.best.h5
230/230 [==============================] - 5s 20ms/step - loss: 0.3658 - accuracy: 0.8490
Epoch 2/10
228/230 [============================>.] - ETA: 0s - loss: 0.1349 - accuracy: 0.9456
Epoch 00002: loss improved from 0.36575 to 0.13465, saving model to /content/drive/My Drive/SP1_2020/HAR with CNN/output.best.h5
230/230 [==============================] - 5s 21ms/step - loss: 0.1346 - accuracy: 0.9457
Epoch 3/10
229/230 [============================>.] - ETA: 0s - loss: 0.1148 - accuracy: 0.9496
Epoch 00003: loss improved from 0.13465 to 0.11560, saving model to /content/drive/My Drive/SP1_2020/HAR with CNN/output.best.h5
230/230 [==============================] - 5s 20ms/step - loss: 0.1156 - accuracy: 0.9494
Epoch 4/10
229/230 [============================>.] - ETA: 

In [0]:
print("\nsample:", testX[0])
print("\nshape: ", testX[0].shape)
print("\nground truth: ", testy[0])



sample: [[ 1.041216   -0.2697959   0.02377977 ...  0.4374637   0.5313492
   0.1365279 ]
 [ 1.041803   -0.280025    0.07629271 ...  0.4682641   0.7210685
   0.09762239]
 [ 1.039086   -0.2926631   0.1474754  ...  0.4982574   0.5203284
   0.08355578]
 ...
 [ 0.9930164  -0.2599865   0.1443951  ... -0.00505586 -0.07734212
   0.03225787]
 [ 0.9932414  -0.2620643   0.1447033  ... -0.02043194 -0.072973
   0.02700848]
 [ 0.9943906  -0.2641348   0.1454939  ... -0.02999741 -0.07064875
   0.03054609]]

shape:  (128, 9)

ground truth:  [0. 0. 0. 0. 1. 0.]


In [0]:
model = tf.keras.models.load_model(dataset_path+'output.best.h5')

In [0]:
t = testX[0]
t = t.reshape(1,128,9)


print("\nclass:",model.predict_classes(t))
print("\nprobs:",model.predict_proba(t))

#print(np.argmax(result))


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).

class: [4]
Instructions for updating:
Please use `model.predict()` instead.

probs: [[1.5009542e-05 1.2962037e-04 2.4740898e-06 4.5978513e-02 9.5386678e-01
  7.6095562e-06]]
